In [1]:
from datasets import load_dataset
from datasets import concatenate_datasets
langs=["cs","de","en","it","nl","sk"]
data_cs= load_dataset("multi_eurlex", language="cs",split='train')
data_de= load_dataset("multi_eurlex", language="de",split='train')
data_en= load_dataset("multi_eurlex", language="en",split='train')
data_it= load_dataset("multi_eurlex", language="it",split='train')
data_nl= load_dataset("multi_eurlex", language="nl",split='train')
data_sk= load_dataset("multi_eurlex", language="sk",split='train')
data_test_cs=load_dataset("multi_eurlex", language="cs",split='test')
data_test_sk=load_dataset("multi_eurlex", language="sk",split='test')


data_train = concatenate_datasets([data_de, data_en, data_it, data_nl,data_cs, data_sk])  
data_test = concatenate_datasets([data_test_sk,data_test_cs])        

Using custom data configuration default-language=cs
Reusing dataset multi_eurlex (/home/ubuntu/.cache/huggingface/datasets/multi_eurlex/default-language=cs/1.0.0/8ec8b79877a517369a143ead6679d1788d13e51cf641ed29772f4449e8364fb6)
Using custom data configuration default-language=de
Reusing dataset multi_eurlex (/home/ubuntu/.cache/huggingface/datasets/multi_eurlex/default-language=de/1.0.0/8ec8b79877a517369a143ead6679d1788d13e51cf641ed29772f4449e8364fb6)
Using custom data configuration default-language=en
Reusing dataset multi_eurlex (/home/ubuntu/.cache/huggingface/datasets/multi_eurlex/default-language=en/1.0.0/8ec8b79877a517369a143ead6679d1788d13e51cf641ed29772f4449e8364fb6)
Using custom data configuration default-language=it
Reusing dataset multi_eurlex (/home/ubuntu/.cache/huggingface/datasets/multi_eurlex/default-language=it/1.0.0/8ec8b79877a517369a143ead6679d1788d13e51cf641ed29772f4449e8364fb6)
Using custom data configuration default-language=nl
Reusing dataset multi_eurlex (/home/

In [2]:
training_data=data_train.to_pandas()
training_data.head()

,celex_id,text,labels
0,32006D0213,ENTSCHEIDUNG DER KOMMISSION\nvom 6. März 2006\...,"[1, 20, 7, 3, 0]"
1,32003R1330,Verordnung (EG) Nr. 1330/2003 der Kommission\n...,"[2, 17]"
2,32003R1786,Verordnung (EG) Nr. 1786/2003 des Rates\nvom 2...,"[3, 19, 6]"
3,31985R2590,*****\nVERORDNUNG (EWG) Nr. 2590/85 DER KOMMIS...,"[12, 17, 19, 6]"
4,31993R1103,VERORDNUNG (EWG) Nr. 1103/93 DER KOMMISSION vo...,"[18, 3, 4, 1]"


In [3]:
type(training_data)

pandas.core.frame.DataFrame

In [22]:
import json
from random import shuffle
LABEL_MAP=[0,1,2]  ##level 1 labels

In [5]:
from torch import nn
from torch.nn import functional as F
from torch.utils.data import Dataset, TensorDataset, DataLoader, RandomSampler
from torch.optim import Adam
from torch.nn import CrossEntropyLoss
from transformers import BertForSequenceClassification
from copy import deepcopy
import gc
import torch
from sklearn.metrics import accuracy_score, confusion_matrix
import numpy as np

class Learner(nn.Module):
    """
    Meta Learner
    """
    def __init__(self, args):
        """
        :param args:
        """
        super(Learner, self).__init__()
        
        self.num_labels = args.num_labels
        self.outer_batch_size = args.outer_batch_size
        self.inner_batch_size = args.inner_batch_size
        self.outer_update_lr  = args.outer_update_lr
        self.inner_update_lr  = args.inner_update_lr
        self.inner_update_step = args.inner_update_step
        self.inner_update_step_eval = args.inner_update_step_eval
        self.bert_model = args.bert_model
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        
        self.model = BertForSequenceClassification.from_pretrained(self.bert_model, num_labels = self.num_labels)
        self.outer_optimizer = Adam(self.model.parameters(), lr=self.outer_update_lr)
        self.model.train()

    def forward(self, batch_tasks, training = True):
        """
        batch = [(support TensorDataset, query TensorDataset),
                 (support TensorDataset, query TensorDataset),
                 (support TensorDataset, query TensorDataset),
                 (support TensorDataset, query TensorDataset)]
        
        # support = TensorDataset(all_input_ids, all_attention_mask, all_segment_ids, all_label_ids)
        """
        task_accs = []
        sum_gradients = []
        num_task = len(batch_tasks)
        num_inner_update_step = self.inner_update_step if training else self.inner_update_step_eval

        for task_id, task in enumerate(batch_tasks):
            support = task[0]
            query   = task[1]
            
            fast_model = deepcopy(self.model)
            fast_model.to(self.device)
            support_dataloader = DataLoader(support, sampler=RandomSampler(support),
                                            batch_size=self.inner_batch_size)
            
            inner_optimizer = Adam(fast_model.parameters(), lr=self.inner_update_lr)
            fast_model.train()
            
            print('----Task',task_id, '----')
            for i in range(0,num_inner_update_step):
                all_loss = []
                for inner_step, batch in enumerate(support_dataloader):
                    
                    batch = tuple(t.to(self.device) for t in batch)
                    input_ids, attention_mask, segment_ids, label_id = batch
                    outputs = fast_model(input_ids, attention_mask, segment_ids, labels = label_id)
                    
                    loss = outputs[0]              
                    loss.backward()
                    inner_optimizer.step()
                    inner_optimizer.zero_grad()
                    
                    all_loss.append(loss.item())
                
                if i % 4 == 0:
                    print("Inner Loss: ", np.mean(all_loss))

            query_dataloader = DataLoader(query, sampler=None, batch_size=len(query))
            query_batch = iter(query_dataloader).next()
            query_batch = tuple(t.to(self.device) for t in query_batch)
            q_input_ids, q_attention_mask, q_segment_ids, q_label_id = query_batch
            q_outputs = fast_model(q_input_ids, q_attention_mask, q_segment_ids, labels = q_label_id)
            
            if training:
                q_loss = q_outputs[0]
                q_loss.backward()
                fast_model.to(torch.device('cpu'))
                for i, params in enumerate(fast_model.parameters()):
                    if task_id == 0:
                        sum_gradients.append(deepcopy(params.grad))
                    else:
                        sum_gradients[i] += deepcopy(params.grad)

            q_logits = F.softmax(q_outputs[1],dim=1)
            pre_label_id = torch.argmax(q_logits,dim=1)
            pre_label_id = pre_label_id.detach().cpu().numpy().tolist()
            q_label_id = q_label_id.detach().cpu().numpy().tolist()
            
            acc = accuracy_score(pre_label_id,q_label_id)
            conf= confusion_matrix(pre_label_id,q_label_id)
            print(conf)
            task_accs.append(acc)
            
            del fast_model, inner_optimizer
            torch.cuda.empty_cache()
        
        if training:
            # Average gradient across tasks
            for i in range(0,len(sum_gradients)):
                sum_gradients[i] = sum_gradients[i] / float(num_task)

            #Assign gradient for original model, then using optimizer to update its weights
            for i, params in enumerate(self.model.parameters()):
                params.grad = sum_gradients[i]

            self.outer_optimizer.step()
            self.outer_optimizer.zero_grad()
            
            del sum_gradients
            gc.collect()
        
        return np.mean(task_accs)

In [6]:
import time, os, random
def random_seed(value):
    torch.backends.cudnn.deterministic=True
    torch.manual_seed(value)
    torch.cuda.manual_seed(value)
    np.random.seed(value)
    random.seed(value)

In [7]:
def create_batch_of_tasks(taskset, is_shuffle = True, batch_size = 4):
    idxs = list(range(0,len(taskset)))
    #idxs = list(range(0,512))
    if is_shuffle:
        random.shuffle(idxs)
    for i in range(0,len(idxs), batch_size):
        yield [taskset[idxs[i]] for i in range(i, min(i + batch_size,len(taskset)))]

In [23]:
class TrainingArgs:
    def __init__(self):
        self.num_labels = 3
        self.meta_epoch=10
        self.k_spt=100
        self.k_qry=30
        self.outer_batch_size = 20    #
        self.inner_batch_size = 50
        self.outer_update_lr = 0.0005  #decrease
        self.inner_update_lr = 0.0001
        self.inner_update_step = 10
        self.inner_update_step_eval = 40
        self.bert_model = 'xlm-roberta-base'
        self.num_task_train = 500
        self.num_task_test = 5

args = TrainingArgs()

In [24]:
data_train_labels0 = data_train.filter(lambda x: (x["labels"][0] == 0  ))
data_train_labels1 = data_train.filter(lambda x: (x["labels"][0] == 1  ))
data_train_labels2 = data_train.filter(lambda x: (x["labels"][0] == 2  ))

print(len(data_train_labels1))

Loading cached processed dataset at /home/ubuntu/.cache/huggingface/datasets/multi_eurlex/default-language=de/1.0.0/8ec8b79877a517369a143ead6679d1788d13e51cf641ed29772f4449e8364fb6/cache-36aa696a219cfefb.arrow
Loading cached processed dataset at /home/ubuntu/.cache/huggingface/datasets/multi_eurlex/default-language=de/1.0.0/8ec8b79877a517369a143ead6679d1788d13e51cf641ed29772f4449e8364fb6/cache-62e76f1ce8b8a142.arrow
Loading cached processed dataset at /home/ubuntu/.cache/huggingface/datasets/multi_eurlex/default-language=de/1.0.0/8ec8b79877a517369a143ead6679d1788d13e51cf641ed29772f4449e8364fb6/cache-9e4db2e8b6905f7f.arrow


9089


In [25]:
data_test_labels0 = data_test.filter(lambda x: (x["labels"][0] == 0  ))
data_test_labels1 = data_test.filter(lambda x: (x["labels"][0] == 1  ))
data_test_labels2 = data_test.filter(lambda x: (x["labels"][0] == 2  ))

print(len(data_test_labels1))

Loading cached processed dataset at /home/ubuntu/.cache/huggingface/datasets/multi_eurlex/default-language=sk/1.0.0/8ec8b79877a517369a143ead6679d1788d13e51cf641ed29772f4449e8364fb6/cache-cf007e649129a915.arrow
Loading cached processed dataset at /home/ubuntu/.cache/huggingface/datasets/multi_eurlex/default-language=sk/1.0.0/8ec8b79877a517369a143ead6679d1788d13e51cf641ed29772f4449e8364fb6/cache-b87557e9f39cd794.arrow


  0%|          | 0/10 [00:00<?, ?ba/s]

322


In [26]:
test_examples_dataset1 = data_test_labels1.shuffle(seed=666).select(range(len(data_test_labels1)))
test_examples_dataset0 = data_test_labels0.shuffle(seed=666).select(range(len(data_test_labels1)))
test_examples_dataset2 = data_test_labels2.shuffle(seed=666).select(range(len(data_test_labels1)))

Loading cached shuffled indices for dataset at /home/ubuntu/.cache/huggingface/datasets/multi_eurlex/default-language=sk/1.0.0/8ec8b79877a517369a143ead6679d1788d13e51cf641ed29772f4449e8364fb6/cache-568dbf313468cca4.arrow


In [27]:
#type(data_train_labels1)

In [28]:
train_examples_dataset1 = data_train_labels1.shuffle(seed=666).select(range(len(data_train_labels1)))
train_examples_dataset0 = data_train_labels0.shuffle(seed=666).select(range(len(data_train_labels1)))
train_examples_dataset2 = data_train_labels2.shuffle(seed=666).select(range(len(data_train_labels1)))



Loading cached shuffled indices for dataset at /home/ubuntu/.cache/huggingface/datasets/multi_eurlex/default-language=de/1.0.0/8ec8b79877a517369a143ead6679d1788d13e51cf641ed29772f4449e8364fb6/cache-ee2fb54945fda7d7.arrow


In [29]:
train_examples = []
test_examples = []

for i in range(len(train_examples_dataset1)):
    train_examples.append(train_examples_dataset1[i])
    train_examples.append(train_examples_dataset0[i])
    train_examples.append(train_examples_dataset2[i])
    
        
for i in range(len(test_examples_dataset1)):
    test_examples.append(test_examples_dataset1[i])
    test_examples.append(test_examples_dataset0[i])
    test_examples.append(test_examples_dataset2[i])
        

In [30]:
print(len(test_examples),len(train_examples))

966 27267


In [33]:
from torch.utils.data import Dataset, TensorDataset
class MetaTask(Dataset):
    
    def __init__(self, examples, num_task, k_support, k_query, tokenizer):
        """
        :param samples: list of samples
        :param num_task: number of training tasks.
        :param k_support: number of support sample per task
        :param k_query: number of query sample per task
        """
        self.examples = examples
        random.shuffle(self.examples)
        
        self.num_task = num_task
        self.k_support = k_support
        self.k_query = k_query
        self.tokenizer = tokenizer
        self.max_seq_length = 128       #increase 
        self.create_batch(self.num_task)
    
    def create_batch(self, num_task):
        self.supports = []  # support set
        self.queries = []  # query set
        
        for b in range(num_task):  # for each task
            # 1.select domain randomly
            #domain = random.choice(self.examples)['domain']   #domain corresponds to low resource language
            #domainExamples = [e for e in self.examples if e['domain'] == domain] 
            #domainExamples=random.choice(self.examples)
            domainExamples=random.sample(self.examples,200)
            # 1.select k_support + k_query examples from domain randomly
            selected_examples = random.sample(list(domainExamples),self.k_support + self.k_query)
            random.shuffle(selected_examples)
            exam_train = selected_examples[:self.k_support]
            exam_test  = selected_examples[self.k_support:]
            
            self.supports.append(exam_train)
            self.queries.append(exam_test)

    def create_feature_set(self,examples):
        all_input_ids      = torch.empty((len(examples), self.max_seq_length), dtype = torch.long)
        all_attention_mask = torch.empty((len(examples), self.max_seq_length), dtype = torch.long)
        all_segment_ids    = torch.empty((len(examples), self.max_seq_length), dtype = torch.long)
        all_label_ids      = torch.empty(len(examples), dtype = torch.long)

        for id_,example in enumerate(examples):
            input_ids = tokenizer.encode(example['text'][:self.max_seq_length])
            attention_mask = [1] * len(input_ids)
            segment_ids    = [0] * len(input_ids)

            while len(input_ids) < self.max_seq_length:
            #while len(input_ids) < len(example['text']):
                input_ids.append(0)
                attention_mask.append(0)
                segment_ids.append(0)

            #label_id = LABEL_MAP[example['label']].  ##check labels
            label_id = LABEL_MAP[example['labels'][0]]
            all_input_ids[id_] = torch.Tensor(input_ids).to(torch.long)
            all_attention_mask[id_] = torch.Tensor(attention_mask).to(torch.long)
            all_segment_ids[id_] = torch.Tensor(segment_ids).to(torch.long)
            all_label_ids[id_] = torch.Tensor([label_id]).to(torch.long)

        tensor_set = TensorDataset(all_input_ids, all_attention_mask, all_segment_ids, all_label_ids)  
        return tensor_set
    
    def __getitem__(self, index):
        support_set = self.create_feature_set(self.supports[index])
        query_set   = self.create_feature_set(self.queries[index])
        return support_set, query_set

    def __len__(self):
        # as we have built up to batchsz of sets, you can sample some small batch size of sets.
        return self.num_task

In [34]:
from transformers import BertModel, AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('xlm-roberta-base', do_lower_case = True)
train = MetaTask(train_examples, num_task = 100, k_support=100, k_query=30, tokenizer = tokenizer)
train[0]

(<torch.utils.data.dataset.TensorDataset at 0x7f41277fdf50>,
 <torch.utils.data.dataset.TensorDataset at 0x7f4124147f90>)

In [ ]:
learner = Learner(args)
    
test = MetaTask(test_examples, num_task = args.num_task_test, k_support=args.k_spt,k_query=args.k_qry, tokenizer = tokenizer)

global_step = 0
#for epoch in range(args.epoch):
for epoch in range(args.meta_epoch):
        train = MetaTask(train_examples, num_task = args.num_task_train, k_support=args.k_spt, 
                         k_query=args.k_qry, tokenizer = tokenizer)

        db = create_batch_of_tasks(train, is_shuffle = True, batch_size = args.outer_batch_size)

        for step, task_batch in enumerate(db):

            acc = learner(task_batch)

            print('Step:', step, '\ttraining Acc:', acc)

            if global_step % 10 == 0:
                random_seed(123)
                print("\n-----------------Testing Mode-----------------\n")
                db_test = create_batch_of_tasks(test, is_shuffle = False, batch_size = 1)
                acc_all_test = []

                for test_batch in db_test:
                    acc = learner(test_batch, training = False)
                    acc_all_test.append(acc)

                print('Step:', step, 'Test F1:', np.mean(acc_all_test))

                random_seed(int(time.time() % 10))

            global_step += 1
        

You are using a model of type xlm-roberta to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.
Some weights of the model checkpoint at xlm-roberta-base were not used when initializing BertForSequenceClassification: ['roberta.encoder.layer.2.attention.self.query.weight', 'roberta.encoder.layer.1.output.dense.weight', 'roberta.encoder.layer.8.attention.self.key.bias', 'roberta.encoder.layer.5.output.dense.bias', 'lm_head.dense.weight', 'roberta.encoder.layer.2.attention.self.key.bias', 'roberta.encoder.layer.8.attention.self.value.weight', 'roberta.encoder.layer.2.output.LayerNorm.bias', 'roberta.encoder.layer.10.attention.output.LayerNorm.weight', 'roberta.encoder.layer.11.output.dense.bias', 'roberta.embeddings.word_embeddings.weight', 'roberta.encoder.layer.7.output.dense.bias', 'roberta.encoder.layer.3.attention.self.key.weight', 'roberta.encoder.layer.5.attention.self.value.weight', 'roberta.embeddings.token_type_embedding

----Task 0 ----
Inner Loss:  1.967118501663208
Inner Loss:  1.1413964629173279
Inner Loss:  1.1446737051010132
[[ 1  2  1]
 [ 0  0  0]
 [11  5 10]]
----Task 1 ----
Inner Loss:  1.6517819166183472
Inner Loss:  1.1064550280570984
Inner Loss:  1.1046413779258728
[[ 0  0  0]
 [13 11  6]
 [ 0  0  0]]
----Task 2 ----
Inner Loss:  1.3573737740516663
Inner Loss:  1.094359576702118
Inner Loss:  1.0915486216545105
[[0 0 0]
 [7 5 6]
 [4 5 3]]
----Task 3 ----
Inner Loss:  1.66815584897995
Inner Loss:  1.1100044250488281
Inner Loss:  1.0742945075035095
[[ 0  0  0]
 [ 1  0  3]
 [10  8  8]]
----Task 4 ----
Inner Loss:  2.0182148814201355
Inner Loss:  1.1265615820884705
Inner Loss:  1.10004460811615
[[ 0  0  0]
 [ 0  1  0]
 [ 5 12 12]]
----Task 5 ----
Inner Loss:  2.113706350326538
Inner Loss:  1.117698609828949
Inner Loss:  1.1131379008293152
[[ 1  0  0]
 [ 0  0  0]
 [14  9  6]]
----Task 6 ----
Inner Loss:  1.2059524655342102
Inner Loss:  1.1052802205085754
Inner Loss:  1.1209802031517029
[[8 7 5]
 [